In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    max_error
)


def plot_interpolation_comparison(
    baseline_df: pd.DataFrame,
    evaluation_df: pd.DataFrame,
    interpolated_file: str = None  # new optional parameter
):
    # === 1. Data preparation ===
    baseline_df = baseline_df.copy()
    evaluation_df = evaluation_df.copy()

    baseline_df['Timestamp'] = pd.to_datetime(baseline_df['Timestamp'])
    evaluation_df['Timestamp'] = pd.to_datetime(evaluation_df['Timestamp'])

    # Convert timestamps to seconds (common origin)
    t0 = baseline_df['Timestamp'].min()
    baseline_df['Seconds'] = (baseline_df['Timestamp'] - t0).dt.total_seconds()
    evaluation_df['Seconds'] = (evaluation_df['Timestamp'] - t0).dt.total_seconds()

    # === 2. Linear interpolation ===
    interp_func = interp1d(
        evaluation_df['Seconds'].values,
        evaluation_df['Metric Value'].values,
        kind='linear',
        bounds_error=False,
        fill_value='extrapolate'
    )
    baseline_df['Interpolated Eval'] = interp_func(baseline_df['Seconds'].values)

    # Optional CSV save
    if interpolated_file:
        baseline_df.to_csv(interpolated_file, index=False)
        print(f"\n💾 CSV file saved: {interpolated_file}")

    # === 3. Metrics evaluation ===
    eval_metrics(baseline_df['Metric Value'].values, baseline_df['Interpolated Eval'].values)

    precision = precision_benchmark_aligned(
        baseline_df[['Timestamp', 'Metric Value']].copy(),
        baseline_df[['Timestamp', 'Interpolated Eval']].copy(),
        tolerance=0.01
    )
    print(f"Precision (±1%)         : {precision:.2f}%")

    # === 4. Plot on a single graph ===
    plt.figure(figsize=(16, 6))
    plt.plot(evaluation_df['Seconds'], evaluation_df['Metric Value'], 'o', label='Dynamic Group', color='tab:orange')
    plt.plot(baseline_df['Seconds'], baseline_df['Interpolated Eval'], '--', label='Interpolated Group (linear)', color='tab:green')
    plt.plot(baseline_df['Seconds'], baseline_df['Metric Value'], '--', label='Baseline Group', color='tab:blue')

    plt.xlabel('Time (s)')
    plt.ylabel('Metric Value')
    plt.title('Comparison: Baseline vs Interpolated vs Dynamic')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
